# 2018 UAV Challenge Medical Express -- Software-In-The-Loop Simulation

The [Rhinohawk](http://www.nova-labs.org/blog/category/groups/rhinohawk-uav/) team at [NovaLabs](http://www.nova-labs.org/blog/) is preparing to enter the [2018 Medical Express Challenge](https://uavchallenge.org/medical-express/).  In this competition an Unmanned Aerial Vehicle ([UAV](https://en.wikipedia.org/wiki/Unmanned_aerial_vehicle)) must find and render assistance to a person stranded the outback.  Teams construct the UAV and AI systems to complete the challenge.  Not an easy task!

The scenario from the challenge website:

> Outback Joe is at his property in remote Queensland, Australia and has been feeling unwell. He has had a remote consultation with his doctor using video conferencing, and his doctor has requested some blood tests be done as soon as possible. Joe is well prepared, has a home sample taking kit, and has taken his blood sample. The challenge is now to get the blood sample to the lab. Joe’s property is very remote and to make matters worse, it has been cut off by floodwaters.

>Teams are invited to attempt to retrieve a blood sample from the remote Outback Joe and return it to base where medical staff will be waiting to analyse it. Teams are encouraged to develop systems that can carry out the mission in a fully autonomous manner using Type 2 Autonomy.
    
## Finding Outback Joe

One part of solving the Challenge is to estimate Outback Joe's location in GPS cordinates.  These coordinates are used to land the UAV close to Joe.  Outback Joe's location is estimated by finding Joe in images from the UAV's camera and translating from image position to GPS coordinates.  The translation to GPS coordinates is accomplished by combining Joe's location in the image, the attitude or POSE of the UAV, and the on-board GPS sensor's location.

There are many pieces needed to locate Joe -- AI software and a custom built UAV.  Software and hardware need to work together seamlessly.  Early in this project that's not going to happen.  One tactic used to ease software development is to simlulate the hardware. This makes software development easier since hardware issues are removed.  The following describes the SILT simulation we used to facilitate developing our Challenge solution.

## SILT

The Rhinohawk team's configuration for the Challenge is a Quadcopter with a camera attached.  The Quadcopter is controlled by a PX4 microprocessor autopilot that can operate in joystick controlled and autonomous modes.  The SILT simultaion compiles the PX4 firmware for a Linux system and uses [Gazebo](http://gazebosim.org/) to simulate the Quadcopter and the sensors attached to the quadcopter.  The image pipeline is implemented in [ROS](http://www.ros.org/) and interacts with the PX4 via [Mavros](http://wiki.ros.org/mavros).  The entire SILT simulation runs on one laptop.

The simulation is based on the PX4 SILT simulation has a Quadcopter simulator among other vehicle models.  We added a camera to the PX4 Iiris Quadcopetr model.  We added a large blue box as our search target instead of Joe We also added a gas station for context:

gas station

The blue box was located on the ground plane at fixed location (3, 3).  The goal of the simultaion is to fly the quadcopter around and process images from the camera.  If all goes well we will predict the box's location from the images and the predicted location will match with the known location.

## Image Pipleline









